In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import nengo
import nengolib
import numpy as np
import pandas as pd
import scipy as sp

In [47]:
from sklearn.decomposition import PCA
data = pd.read_csv("pinsoro-2017-06-01-102743523980-small.csv", low_memory=False)

dt = 0.001

def extract_pattern(start, end):
    pattern = np.array(data.iloc[start:end,11:195]).astype(float)
    frames = np.array(data.iloc[start:end,9]).astype(int)

    good_indices = frames != -1
    frames = frames[good_indices]
    pattern = pattern[good_indices]

    fps = 30.0
    t_sample = (frames - frames[0])/fps

    t = np.arange(int(t_sample[-1]/dt))*dt
    
    result = []
    for i in range(pattern.shape[1]):
        p = np.interp(t, t_sample, pattern[:,i])
        #bad_indexes = np.isnan(p)
        #good_indexes = np.logical_not(bad_indexes)
        #good_pattern = p[good_indexes]
        #interpolated = np.interp(bad_indexes.nonzero(), good_indexes.nonzero(), p[:,1])
        #p[bad_indexes] = interpolated
        result.append(p)
    result = np.array(result).T
    
    return t, result


t1, result1 = extract_pattern(2977, 3379)
t2, result2 = extract_pattern(3702, 4104)

#np.savetxt('result.csv', result1, delimiter=',')


In [11]:
nb_components=1
def extract_component(data):
    pca_model=PCA(n_components=nb_components).fit(data)
    p = pca_model.transform(data)
    print("Cumulative explained variance: %s" % pca_model.explained_variance_ratio_.cumsum())
    
    return p

p1 = extract_component(result1)
p2 = extract_component(result2)

plt.plot(t1, p1)
plt.plot(t2, p2)

TypeError: float() argument must be a string or a number, not 'interp1d'

Pattern 1 is goal oriented play. 
Pattern 2 is no-play. 

Now let's create a network that represents a rolling window in time (Aaron's "delay network"). The process determines what sort of pattern the network will be optimized for -- here we just go with white noise of a maximum of 3Hz.  theta determines how big the rolling window is -- here we use 0.5 seconds.

In [ ]:
theta = 0.402

net = nengo.Network()
with net:
    process = nengo.processes.WhiteSignal(period=100., high=3.0, y0=0)
    rw = nengolib.networks.RollingWindow(theta=theta, n_neurons=3000, process=process, neuron_type=nengo.LIFRate(tau_rc=0.02, tau_ref=0.002, amplitude=1))

Now we need to create the training data for decoding out of the rolling window. Our patterns are larger than the rolling window, so to create our training data we will take our patterns, shift them, and cut them down to the right size. In order to then give that to nengo, we also need to project from the window's space to the internal representation space (using the inv_basis).

The target array is the desired output value for each of the slices of the pattern in eval_points. We'll use 1 for pattern1 and -1 for pattern2.

In [ ]:
s_window = int(theta/dt)
s_pattern = min(len(p1), len(p2))

t_window = np.linspace(0, 1, s_window)
inv_basis = rw.inverse_basis(t_window)
    
eval_points=[]
target = []
for i in range(s_pattern):
    eval_points.append(np.dot(inv_basis, np.roll(p1, i)[:s_window]))
    target.append([1])
    eval_points.append(np.dot(inv_basis, np.roll(p2, i)[:s_window]))
    target.append([-1])
    
eval_points = np.array(eval_points)
print(eval_points.shape)

Now we can create a connection optimized to do this decoding

In [ ]:
with net:
    result = nengo.Node(None, size_in=1)
    nengo.Connection(rw.state, result,
                     eval_points=eval_points, scale_eval_points=True,
                     function=target, synapse=0.1)

Let's try feeding in those two patterns and see what the response is

In [ ]:
p_all = np.hstack([p1, p2])

model = nengo.Network()
model.networks.append(net)
with model:    
    stim = nengo.Node(nengo.processes.PresentInput(p_all, presentation_time=0.015))
    nengo.Connection(stim, rw.input, synapse=None)
    
    p_result = nengo.Probe(result)
    p_stim = nengo.Probe(stim)
sim = nengo.Simulator(model)
sim.run(30)

plt.plot(sim.trange(), sim.data[p_stim], label='input')
plt.plot(sim.trange(), sim.data[p_result], label='output')
plt.legend(loc='best')